In [ ]:
pip install requests

In [ ]:
pip install progress_bar

In [ ]:
pip install selenium

In [ ]:
pip install beautifulsoup4

In [ ]:
# import necessary modules
import time
import datetime
import random
import numpy as np
import base64

import re
import os

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from tqdm import tqdm
import urllib.request 
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.expected_conditions import presence_of_element_located
# browser.quit() # close the whole browser
# browser.close() # close the tab

In [ ]:
def get_product_info_from_mainpage(URL):
    product_links = []
    image_links = []
    prices = []
    
    driver = webdriver.Chrome('C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe')
    driver.implicitly_wait(100)
    
    try:
        driver.get(URL)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(10)
        
        product_cards = soup.find_all('div', {"class" : "ProductCard-container ProductCard-container--contained"})

        print("Number of products scraped from the page: ", len(product_cards))
        
        i = 0
        for product_card in product_cards:
            try:
                #get image link
                driver.implicitly_wait(3)
                image = product_card.find('div', {"class" : "BrowseProductCard-imageWrap"})
                #image_link = image.find_element_by_tag_name('img').get_attribute('src')
                #image_link = image.find('img')["src"]
                #image_link = image.select('img').get('src')
                image_link = image.find('img')["srcset"].split(',')[-1].split(" ")[0]
                image_links.append(image_link)

                
                #get product link
                product_link = product_card.find('a')['href']
                product_links.append(product_link)                


                #get price
                price_block = product_card.find('div', {"class" : "ProductCard-pricing"})
                price = price_block.find('span').text

                '''
                try:
                    price = price_block.find('span', {"class" : "ProductCard-price"}).text
                except:
                    price = price_block.find('span', {"class" : "ProductCard-price ProductCard-price--sale"}).text
                '''
                
                prices.append(price)
                i += 1
                
                #Print summary information
                print(f"Item: {i}\n")
                print(f"Product link: {product_link}\n")
                print(f"Image link: {image_link}\n")
                print(f"Price: {price}\n")
                print("==============================================")
            
            except:
                print("Error")
                pass
        driver.close()
        print("Lengths of lists: ", len(product_links), len(image_links), len(prices))
        return (product_links, image_links, prices)
    
    except TimeoutException as e:
        print("Page load Timeout Occured. Quitting !!!")
        driver.close()

'''
def get_info_from_productlink(product_link):  #get the first image
    info = dict()
    page = requests.get(product_link)
    driver = webdriver.Chrome('C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe')
    driver.get(product_link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(random.randint(3,3))

    #price_block = soup.find('div', {"class" : "BasePriceBlock BasePriceBlock--highlight"})
    #price_block = soup.find('div', {"class" : "BasePriceBlock"})
    price_block = soup.find('span', {"class" : "notranslate"})
    price = price_block.text
    info['price'] = price
    #print(price)   
    
    image = soup.find('div', {'class' : "ImageComponent ImageComponent--overlay"})
    image_link = image.find('img')['src']
    info['img_url'] = image_link
    
    #print(f"Image link: {image_link}\n")
    #print(f"Link to product: {product_link}\n")

    driver.close()
    #browser.switch_to.window(browser.window_handles[0])
    return info
'''

In [ ]:
#TEST
get_product_info_from_mainpage('https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=96&curpage=2')

In [ ]:
def save_data(product_info_list, i):
    result = [[product[0], product[1]['price'], product[1]['img_url']] for product in product_info_list]
    df = pd.DataFrame(result)
    now = str(datetime.datetime.now()).replace(" ","").replace(":","").replace("-","").split(".")[0][4:][:-2]
    df.to_csv(f'sofa_info_{str(i)}.csv', index=False, header=['item_no', 'price', 'img_url'])
    
def save_image_from_url(product, category):    
    bar_wrap = [None]

    item_no = product.name + 5780
    print("Product Row: ", product.name)
    base_name = "C:/Users/Brian/Desktop/ThePriceIsRight/" + category + "/"
    filename = f"./{category}/{item_no}.jpeg"
    filepath = base_name + str(item_no) + ".jpg"
    url = str(product['img_url'])
    
    try:
        print(url)
        urllib.request.urlretrieve(url, filepath)
        return filename
    except:
        print(f"Failed to download image:{url}\n")
        return f"Failed to download image: {url}\n"
    
def save_images_from_csv(df):
    tqdm.pandas()
    df['filename'] = df.progress_apply(lambda x: save_image_from_url(x, category = 'sofa'), axis=1)
    return df

In [ ]:
product_info = []
base_url = 'https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=96&curpage='
#base_url = 'https://www.wayfair.com/furniture/sb0/sofas-c413892.html?curpage='

#data from pages 1 - 4
for page in tqdm(range(1,5)):
    url = base_url + str(page)
    (product_links, image_links, prices) = get_product_info_from_mainpage(url)
    print(f"Number of products on Page {page}: {len(product_links)}\n")
    i = 5779
    for (product_url, img_url, price) in tqdm(zip(product_links, image_links, prices)):
        i += 1
        #item_no = f"{page}_{i}"
        item_no = f"{i}"
        try:
            info = {'price' : price, 'img_url' : img_url}
            product_info.append((item_no,info))     
        except:
            print(f"Failed. Product link: {product_url}")
            pass

In [ ]:
save_data(product_info,1)
product_info

In [ ]:
df = pd.read_csv('sofa_info_1.csv')
print(df.shape)
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [ ]:
df_new = save_images_from_csv(df)
df_new.to_csv('sofa.csv', index=False)

In [ ]:
df_new